# Mask R-CNN - Train on Shapes Dataset

### Notes from implementation

This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import  gc
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pprint
import keras.backend as KB
sys.path.append('../')

import mrcnn.model     as modellib
import mrcnn.visualize as visualize
import mrcnn.shapes    as shapes
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 

from mrcnn.utils       import stack_tensors, stack_tensors_3d
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.visualize   import plot_gaussian
# from mrcnn.pc_layer    import PCTensor
# from mrcnn.pc_layer   import PCNLayer

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_logs")
# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4)


# Build configuration object -----------------------------------------------
config = shapes.ShapesConfig()
config.BATCH_SIZE      = 5                  # Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 5                  # Must match BATCH_SIZE
config.STEPS_PER_EPOCH = 2
config.FCN_INPUT_SHAPE = config.IMAGE_SHAPE[0:2]
config.display() 

# Build shape dataset        -----------------------------------------------
# Training dataset
# generate 500 shapes 
dataset_train = shapes.ShapesDataset()
dataset_train.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
dataset_val = shapes.ShapesDataset()
dataset_val.load_shapes(50, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_val.prepare()

try :
    del model, train_generator, val_generator, mm
    gc.collect()
except: 
    pass
# Load and display random samples
# image_ids = np.random.choice(dataset_train.image_ids, 3)
# for image_id in [3]:
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)

# Create Model

In [ ]:
try :
    del model
    gc.collect()
except: 
    pass

model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
#model.keras_model.summary(line_length = 120) 

# Which weights to start with?
init_with = "last"  # imagenet, coco, or last
if init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that are different due to the different number of classes
    # See README for instructions to download the COCO weights
    loc=model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    loc= model.load_weights(model.find_last()[1], by_name=True)

model.compile_only(learning_rate=config.LEARNING_RATE, layers='heads')
KB.set_learning_phase(1)

###  Print some model information 

In [ ]:
# print('\n Learning phase values is L ' ,KB.learning_phase())
# print('\n Metrics (_get_deduped_metrics_names():) ') 
# pp.pprint(model.keras_model._get_deduped_metrics_names())
# print('\n Outputs: ') 
# pp.pprint(model.keras_model.outputs)
# print('\n Losses (model.metrics_names): ') 
# pp.pprint(model.keras_model.metrics_names)

model.keras_model.summary(line_length = 150) 

### Define Data Generator

In [ ]:
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=model.config.BATCH_SIZE,
                                 augment = False)
val_generator = data_generator(dataset_val, model.config, shuffle=True, 
                                batch_size=model.config.BATCH_SIZE,
                                augment=False)

### Get next shapes from generator and display loaded shapes

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

### Display loaded shapes

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)



### Push Data thru model using get_layer_output()

In [ ]:

layers_out = get_layer_output_2(model.keras_model, train_batch_x, 1)


### Display first two images from training batch 

In [ ]:
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]
img         = 2
image_id = img_meta[img,0]
print('Image id: ',image_id)

image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

img = 3
image_id = img_meta[img,0]
print('Image id: ',image_id)
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


## Plot Predicted and Ground Truth Probability Heatmaps `pred_gaussian` and `gt_gaussian` (Tensorflow)
`gt_gaussian` and `gt_gaussian2` from Numpy and Tensorflow PCN layers

In [ ]:
gt_heatmap  = layers_out[27]     # gt_gaussiam 
pred_heatmap= layers_out[24]  # pred_gaussian

print('gt_gaussian heatmap shape : ', gt_heatmap.shape, ' pred_gaussian heatmap shape: ', pred_heatmap.shape)
num_images = 1 # config.IMAGES_PER_GPU
num_classes = config.NUM_CLASSES

img = 2

image_id = img_meta[img,0]
print('Image id: ',image_id)
print('Classes (1: circle, 2: square, 3: triangle ): ')
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


for cls in range(num_classes):
    ttl = 'GROUND TRUTH HEATMAP - image :  {} class: {} '.format(img,cls)
    print(' *** Zout  ', gt_heatmap[img,:,:,cls].shape, ttl)   
    plot_gaussian( gt_heatmap[img,:,:,cls], title = ttl)
    
    ttl = 'PREDICTED heatmap  - image :  {} class: {} '.format(img,cls)     
    print(' *** pred_heatmap ', pred_heatmap[img,:,:,cls].shape, ttl)   
    plot_gaussian(pred_heatmap[img,:,:,cls], title = ttl)  



## Predicted and Ground Truth Probability Heatmaps `pred_gaussian` and `gt_gaussian`  (NUMPY)
`gt_gaussian` and `gt_gaussian2` from Numpy and Tensorflow PCN layers

In [ ]:
gt_heatmap  = layers_out[21]     # gt_gaussiam 
pred_heatmap= layers_out[18]  # pred_gaussian

print('gt_gaussian heatmap shape : ', gt_heatmap.shape, ' pred_gaussian heatmap shape: ', pred_heatmap.shape)
num_images = 1 # config.IMAGES_PER_GPU
num_classes = config.NUM_CLASSES

img = 2

image_id = img_meta[img,0]
print('Image id: ',image_id)
print('Classes (1: circle, 2: square, 3: triangle ): ')
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


for cls in range(num_classes):
    ttl = 'GROUND TRUTH HEATMAP - image :  {} class: {} '.format(img,cls)
    print(' *** Zout  ', gt_heatmap[img,:,:,cls].shape, ttl)   
    plot_gaussian( gt_heatmap[img,:,:,cls], title = ttl)
    
    ttl = 'PREDICTED heatmap  - image :  {} class: {} '.format(img,cls)     
    print(' *** pred_heatmap ', pred_heatmap[img,:,:,cls].shape, ttl)   
    plot_gaussian(pred_heatmap[img,:,:,cls], title = ttl)  


## Plot Predicition Probability Heatmaps `pred_gaussian` and `pred_gaussian2` (Numpy vs. Tensorflow)

`pred_gaussian` and `pred_gaussian2` from Numpy and Tensorflow layers 

In [ ]:
# pred_heatmap_np = layers_out[18]  # pred_gaussiam 
# pred_heatmap_tf = layers_out[24]  # pred_gaussian2
pred_heatmap_np = layers_out[21]  # pred_gaussiam 
pred_heatmap_tf = layers_out[27]  # pred_gaussian2

print('pred_gaussian_np heatmap shape : ', pred_heatmap_np.shape, ' pred_gaussian_tf heatmap shape: ', pred_heatmap_tf.shape)
num_images = 1 # config.IMAGES_PER_GPU
num_classes = config.NUM_CLASSES

img = 2

image_id = img_meta[img,0]
print('Image id: ',image_id)
print('Classes (1: circle, 2: square, 3: triangle ): ')
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


for cls in range(num_classes):
    ttl = 'NUMPY predicted HEATMAP - image :  {} class: {} '.format(img,cls)
    print(' *** Zout  ', pred_heatmap_np[img,:,:,cls].shape, ttl)   
    plot_gaussian( pred_heatmap_np[img,:,:,cls], title = ttl)
    
    ttl = 'PREDICTED heatmap  - image :  {} class: {} '.format(img,cls)     
    print(' *** pred_heatmap ', pred_heatmap_tf[img,:,:,cls].shape, ttl)   
    plot_gaussian(pred_heatmap_tf[img,:,:,cls], title = ttl)  
  


## Plot Output from FCN network `fcn_bilinear` and compare with `pred_gaussian`


In [ ]:
from mrcnn.visualize import plot_gaussian
Zout  = layers_out[7]     # gt_gaussiam 
Zout2 = layers_out[18]    # gt_gaussian2

print(Zout.shape, Zout2.shape)
num_images = config.IMAGES_PER_GPU
num_classes = config.NUM_CLASSES

img = 0
for cls in range(num_classes):
    ttl = 'NUMPY - image :  {} class: {} '.format(img,cls)
    print(' *** Zout  ', Zout[img,:,:,cls].shape, ttl)   
    plot_gaussian( Zout[img,:,:,cls], title = ttl)
    
    ttl = 'TENSORFLOW - image :  {} class: {} '.format(img,cls)     
    print(' *** Zout2 ', Zout2[img,:,:,cls].shape, ttl)   
    plot_gaussian(Zout2[img,:,:,cls], title = ttl)  


### Display ground truth bboxes from Shapes database (using `load_image_gt` )

Here we are displaying the ground truth bounding boxes as provided by the dataset

In [ ]:
img = 2
image_id = img_meta[img,0]
print('Image id: ',image_id)
p_original_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
# print(p_gt_class_id.shape, p_gt_bbox.shape, p_gt_mask.shape)
print(p_gt_bbox)
print(p_gt_class_id)
visualize.draw_boxes(p_original_image, p_gt_bbox)

image_id = img_meta[img,0]
print('Image id: ',image_id)
p_original_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
# print(p_gt_class_id.shape, p_gt_bbox.shape, p_gt_mask.shape)
print(p_gt_bbox)
print(p_gt_class_id)
visualize.draw_boxes(p_original_image, p_gt_bbox)

### Display Predicted  Ground Truth Bounding Boxes  `gt_tensor` and `gt_tensor2`

layers_out[22]  `gt_tensor` is based on input_gt_class_ids and input_normlzd_gt_boxes
layers_out[28]  `gt_tensor2` is based on input_gt_class_ids and input_normlzd_gt_boxes, generated using Tensorflow

Display the Ground Truth bounding boxes from the tensor we've constructed

In [ ]:
from mrcnn.utils  import stack_tensors, stack_tensors_3d
# print(gt_bboxes)
# visualize.display_instances(p_original_image, p_gt_bbox, p_gt_mask, p_gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))
# pp.pprint(gt_bboxes)
img = 2
image_id = img_meta[img,0]

print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)   
gt_bboxes_stacked = stack_tensors_3d(layers_out[22][img])
print(gt_bboxes_stacked)
visualize.draw_boxes(p_image, gt_bboxes_stacked[:,2:6])


print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)   
gt_bboxes_stacked = stack_tensors_3d(layers_out[28][img])
print(' From pcn_layer_tf (TENSORFLOW)')
print(gt_bboxes_stacked)
visualize.draw_boxes(p_image, gt_bboxes_stacked[:,2:6])

### Display Predicted Bounding Boxes  `pred_tensor` and `pred_tensor2` 

layers_out[19]  `gt_tensor` is based on input_gt_class_ids and input_normlzd_gt_boxes - Using Numpy
layers_out[25]  `gt_tensor` is based on input_gt_class_ids and input_normlzd_gt_boxes - Using Tensorflow

Display the Ground Truth bounding boxes from the tensor we've constructed

In [ ]:
from mrcnn.utils  import stack_tensors, stack_tensors_3d
# print(gt_bboxes)
# visualize.display_instances(p_original_image, p_gt_bbox, p_gt_mask, p_gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))
# pp.pprint(gt_bboxes)
img = 2
image_id = img_meta[img,0]

print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)   
gt_bboxes_stacked = stack_tensors_3d(layers_out[19][img])
print(gt_bboxes_stacked)
visualize.draw_boxes(p_image, gt_bboxes_stacked[:,2:6])


print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)   
gt_bboxes_stacked = stack_tensors_3d(layers_out[25][img])
print(' From pcn_layer_tf (TENSORFLOW)')
print(gt_bboxes_stacked)
visualize.draw_boxes(p_image, gt_bboxes_stacked[:,2:6])

In [ ]:
np.set_printoptions(linewidth=120, precision=5)
print(' gt_cls_cnt from Numpy - shape: ', layers_out[5].shape)
print(layers_out[5])


print(' gt_cls_cnt from TF - shape: ', layers_out[13].shape)
print(layers_out[13])


## Display RoI proposals `pred_bboxes` generated for one class

Display bounding boxes from tensor of proposals produced by the network 
Square: 1 , Circle:2 , Triangle 3

In [ ]:
img = 0
cls = 3 # <==== Class to display

image_id = img_meta[img,0]
print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)

pred_tensor = layers_out[4]
print(pred_tensor[img,cls,:].shape)
#+'-'+str(np.around(int(x[1]),decimals = 3))
# class id: str(int(x[6]))+'-'+
caps = [str(int(x[0]))+'-'+str(np.around(x[1],decimals = 3))  for x in pred_tensor[img,cls,:].tolist() ]
print(caps)

visualize.draw_boxes(p_image, pred_tensor[img,cls,:,2:6], captions = caps)

## Display RoI proposals `pred_bboxes` generated for all classes in image

Display bounding boxes from tensor of proposals produced by the network 
Square: 1 , Circle:2 , Triangle -3

In [ ]:
img = 0
# cls = 1  # <==== Class to dispaly

image_id = img_meta[img,0]
print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)

pred_tensor = layers_out[4][img]
print(pred_tensor[img,cls,:].shape)
pred_bboxes_stacked = stack_tensors_3d(pred_tensor)
# lst2   = [np.squeeze(item) for item in np.split(pred_tensor, pred_tensor.shape[0], axis = 0 )]
# results = np.concatenate( [ i[~np.all(i[:,2:6] == 0, axis=1)] for i in lst2] , axis = 0)
caps = [str(int(x[6]))+'-'+str(int(x[0]))+'-'+str(np.around(x[1],decimals = 3))  for x in pred_bboxes_stacked.tolist() ]
print(caps)

# print(pc_tensor.pred_tensor[1,3,:])
# print(pc_tensor.pred_tensor[1,3,:,2:6])
visualize.draw_boxes(p_image, pred_bboxes_stacked[:,2:6], captions = caps)




#+'-'+str(np.around(int(x[1]),decimals = 3))
# class id: str(int(x[6]))+'-'+
# caps = [str(int(x[0]))+'-'+str(np.around(x[1],decimals = 3))  for x in pred_tensor[img,cls,:].tolist() ]
# print(caps)

# visualize.draw_boxes(p_image, pred_tensor[img,cls,:,2:6], captions = caps)

###  Simulate calculations of mrcnn_bbox_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss

target_class_ids = layers_out[1][0:1]
target_bbox      = layers_out[2][0:1]
mrcnn_bbox       = layers_out[10][0:1]
mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids

print('target_class_ids', target_class_ids.shape)
print(target_class_ids)  # tgt_class_ids
print(' class with max probability', mrcnn_class_ids.shape)
print(mrcnn_class_ids)
print('target_bboxes', target_bbox.shape)
# print(target_bbox)  # tgt_bounding boxes
print('mrcnn_bboxes',mrcnn_bbox.shape)
# print(mrcnn_bbox)  #mrcnn_bboxes
pred_bbox = mrcnn_bbox

# calc mrcnn_bbox_loss
target_class_ids = K.reshape(target_class_ids, (-1,))
print(target_class_ids.shape)
target_bbox      = K.reshape(target_bbox, (-1, 4))
print('target_bboxx: ', target_bbox.shape)
pred_bbox        = K.reshape(pred_bbox, (-1, pred_bbox.shape[2], 4))
print('pred_bbox : ', pred_bbox.shape)

positive_roi_ix        = tf.where(target_class_ids > 0)[:, 0]
print(positive_roi_ix.eval())
positive_roi_class_ids = tf.cast( tf.gather(target_class_ids, positive_roi_ix), tf.int64)
print(positive_roi_class_ids.eval())
indices                = tf.stack([positive_roi_ix, positive_roi_class_ids], axis=1)
print(indices.eval())


target_bbox = tf.gather(target_bbox, positive_roi_ix)
print(target_bbox.eval())
pred_bbox   = tf.gather_nd(pred_bbox, indices)
print(pred_bbox.eval())

print('tf.size ',tf.size(target_bbox).eval())

diff = K.abs(target_bbox - pred_bbox)
print(diff.eval())

less_than_one = K.cast(K.less(diff, 1.0), "float32")
# print(less_than_one.eval())

loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
# print( (1-less_than_one).eval())



# loss        = K.switch(tf.size(target_bbox) > 0,
#                 smooth_l1_loss(y_true=target_bbox, y_pred=pred_bbox),
#                 tf.constant(0.0))
print(loss.eval())
sumloss = K.sum(loss)
print(sumloss.eval())
print((sumloss/40).eval())
meanloss        = K.mean(loss)
print(meanloss.eval())

For each class:
- determine the center of each bounding box.
- center a 2d gaussian distribution with the mean = center of bounding box and sigma = height/width
- place dist on mesh grid
- normalize
- draw heatmap

# Stacking Routines
###  Stacking routine -- break down by images 

In [ ]:
## stack an [Batch x Class x Row x Col] tensor into Row x Cols
##------------------------------------------------------------------
pred_tensor = layers_out[8]
lst2 = [ np.squeeze(item) for item in np.split(pred_tensor, pred_tensor.shape[0], axis = 0 )]
lst2 = [ np.squeeze(np.concatenate(np
                                   .split(item, item.shape[0], axis = 0 ), axis = 1)) for item in lst2]
result = [ item[~np.all(item[:,2:6] == 0, axis=1)] for item in lst2]
print(len(result))
# print(result)

###  Stack tensor routine for one image 

In [ ]:
## stack an  [Class x Row x Col] tensor into Row x Cols
##------------------------------------------------------------------
pred_tensor = layers_out[8][0]
lst2   = [np.squeeze(item) for item in np.split(pred_tensor, pred_tensor.shape[0], axis = 0 )]
result = np.concatenate( [ i[~np.all(i[:,2:6] == 0, axis=1)] for i in lst2] , axis = 0)
print(result.shape)
# print(result)

# Verify matching results between numpy and tensorflow routines
### Control pred_tensor / pred_tensor2 on 100 training shapes

In [ ]:
for i in range(100):
    train_batch_x, train_batch_y = next(train_generator)

    layers_out = get_layer_output_2(model.keras_model, train_batch_x, 1, verbose = False)

    pt   = layers_out[5]   # pred_tensor
    pt2  = layers_out[11]  # pred_tensor_2
    
#     pcc  = layers_out[9]   # pred_cls_cnt
#     pcc2 = layers_out[15]  # pred_cls_cnt_2

#     print( pt2.shape, pcc2.shape)
#     print( pt.shape, pt2.shape)
#     print(pc2)

    for img in range(config.BATCH_SIZE):
        for cls in range(4):
#             print(pt2[img][cls])
#             print(pt[img][cls])
            pt_equal = np.all(pt2[img,cls,:,1:-1]== pt[img,cls,:,1:-1], axis = -1)
            print('* Iteration', i , 'Image ',img,' Class ',cls, ' pred_tesnor == pred_tensor2 : ',pt_equal.all())
            
            if (~pt_equal.all()):
#                 print(' Iteration', i , 'Image ',img,' Class ',cls, ' ALL pt_equal: ',pt_equal.all())
                print(pt_equal)
                print('\n -- using numpy \n',pt[img][cls,~pt_equal,:-1])
                print('\n -- using tensorflow \n',pt2[img][cls,~pt_equal])
                print()
    #             print('\n -- using numpy \n',pt[img][cls])            
    #             print('\n -- using tensorflow \n',pt2[img][cls])
    

### Control pred_tesnor / pred_tensor2

In [ ]:
pt   = layers_out[4]   # pred_tensor
pt2  = layers_out[12] 
 
print( pt.shape, pt2.shape)

for img in range(config.BATCH_SIZE):
    for cls in range(4):
        equal = np.all(pt[img][cls,:,1:7] == pt2[img][cls,:,1:7], axis = -1)
        print('Image ',img,' Class ',cls, ' ALL EQUAL: ',equal.all())
#         print(' numpy results ')
#         print( pt[img,cls])
#         print('tensorflow results ')
#         print(pt2[img,cls])
        if (~equal.all()):
#             print('Image ',img,' Class ',cls, ' ALL EQUAL: ',equal.all())
            print(equal)
            print('\n -- using numpy (pt) \n',pt[img][cls,~equal,:-1])
            print('\n -- using tensorflow (pt2) \n',pt2[img][cls,~equal])
            print()
#             print('\n -- using numpy \n',pt[img][cls])            
#             print('\n -- using tensorflow \n',pt2[img][cls])
    

### Display pred_tensor from Numpy and Tensorflow PCN_Layer 

In [ ]:
# np.set_printoptions(threshold=99999, linewidth=2000)
# print(np.array2string(mask[...,0],max_line_width=2000,separator=''))
pt   = layers_out[4]   # pred_tensor
pt2  = layers_out[12] 
 
print( pt.shape, pt2.shape)
img = 0

for cls in range(4):
#     equal = np.all(pt[img][cls,:,1:7] == pt2[img][cls,:,1:7], axis = -1)
    print('Image ',img,' Class ',cls)
    print(' Output from PCN_Layer (Numpy)')
    print( pt[img,cls])
    print(' Output from PCN_Layer (Tensorflow)')
    print(pt2[img,cls])


###  pred_gaussian / pred_gaussian2  & gt_gaussian / gt_gaussian2

In [ ]:
# pt   = layers_out[4]   # pred_gaussian 
# pt2  = layers_out[10]  # pred_gaussian_2
np.set_printoptions(linewidth=130, threshold=20000)
gt   = layers_out[7]   # gt_gaussian 
gt2  = layers_out[13]  # gt_gaussian_2
# gt   = np.where(gt > 1e-6,gt,0)
# gt2   = np.where(gt2 > 1e-6,gt2,0)
print( ' pt shape ', gt.shape, ' pt2.shape ', gt2.shape)

for img in range(config.BATCH_SIZE):
#     print(' from np ')
#     print(pt[img])
#     print(' from tensorflow')
#     print(pt2[img])
    for cls in range(4):
        equal = np.all(gt2[img, cls] == gt[img, cls], axis=-1)      
        print( 'Image ',img,' Class ',cls, '  all equal: ',equal.all())        
        
        if (~equal.all()):
            print(~equal)
            print( 'Image ',img,' Class ',cls, ' ALL EQUAL: ',equal.all())
#             print('\n -- using numpy      \n',  gt[img, cls, ~equal])
#             print('\n -- using tensorflow \n', gt2[img, cls, ~equal])
# if not equal display the different between the mismatching rows
            for i in range(equal.shape[0]):
                if ~equal[i]:
                    diff = np.abs(gt2[img, cls, i] - gt[img, cls, i])
                    big_error = np.any(diff > 3.0e-9, axis = -1)
                    print('   row = ', i, ' rows equal = ',equal[i], '   Big Error (larger than 7.0e-8): ' ,big_error)
                    if big_error:
                        print(' difference  :', diff )
#                     print(' -- using numpy      \n',gt[img,cls,i])            
#                     print(' -- using tensorflow \n',gt2[img,cls,i])
    